In [1]:
import tonic
from torch.utils.data import DataLoader
import snntorch as snn
from snntorch import spikeplot as splt
from snntorch import spikegen
import numpy as np

import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from tonic import transforms as T
from snntorch import functional as SF

minibatch = 128

#  downsample the data to use it (to make the network and training simpler)
shd_timestep = 1e-6
shd_channels = 700
shd_classes = 20
net_channels = 128 #128
net_dt = 10e-3
sample_T = 250
# - We'll use `tonic` to downsample the data, using a transformation pipeline
transform = T.Compose([
    # - Downsample in time and space
    T.Downsample(
        time_factor=shd_timestep / net_dt,
        spatial_factor=net_channels / shd_channels
        ),

    # - Rasterise the events
    T.ToFrame(
        sensor_size=(net_channels, 1, 1), time_window=1
    ),
    
    # - Convert to a tensor
    torch.Tensor,

    # -samples are not too long in time
    lambda m: torch.squeeze(m)[:sample_T, :],
])

trainset=tonic.datasets.SHD(save_to="./original-data",train=True,transform=transform)
testset=tonic.datasets.SHD(save_to="./original-data",train=False,transform=transform)

train_loader = DataLoader(trainset, batch_size=minibatch, shuffle=True,  drop_last=True, pin_memory=True, collate_fn=tonic.collation.PadTensors(batch_first=True), num_workers=0)
test_loader = DataLoader(testset,   batch_size=minibatch, shuffle=False, drop_last=True, pin_memory=True, collate_fn=tonic.collation.PadTensors(batch_first=True), num_workers=0)

c:\Users\Slugbotics\Documents\Jonathan\.conda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

In [3]:
# Network Architecture
num_inputs = 128
num_hidden = 1000
num_outputs = 20

# Temporal Dynamics
num_steps = 25
beta = 0.95

In [4]:

# Define Network
class RSNN(nn.Module):
    def __init__(self):
        super().__init__()

        num_inputs = 128
        num_hidden = 1000
        num_outputs = 20
        beta = 0.9

        # Initialize layers
        self.fc1 = nn.Linear(num_inputs, num_hidden)
        self.lif1 = snn.RLeaky(beta=beta, linear_features=num_hidden)
        self.fc2 = nn.Linear(num_hidden, num_outputs)
        self.lif2 = snn.RLeaky(beta=beta, linear_features=num_outputs)

    def forward(self, x):

        # Initialize hidden states at t=0
        spk1, mem1 = self.lif1.init_rleaky()
        spk2, mem2 = self.lif2.init_rleaky()
        
        # Record the final layer
        spk2_rec = []
        mem2_rec = []

        for step in range(x.size(0)):
            cur1 = self.fc1(x[step].flatten(1))
            spk1, mem1 = self.lif1(cur1, spk1, mem1)
            cur2 = self.fc2(spk1)
            spk2, mem2 = self.lif2(cur2, spk2, mem2)
            spk2_rec.append(spk2)
            mem2_rec.append(mem2)

        return torch.stack(spk2_rec, dim=0), torch.stack(mem2_rec, dim=0)

        
# Load the network onto CUDA if available
rnet = RSNN().to(device)

In [5]:
def calculate_accuracy(output, target):
    """
    A function for checking acc of LSTM etc.
    """
    import torch
    predicted:torch.Tensor = torch.argmax(output, 1)
    correct = (predicted == target).sum().item()
    accuracy = correct / target.size(0)
    return accuracy

In [6]:
criterion = SF.ce_rate_loss()
optim = torch.optim.Adam(rnet.parameters(), lr=1e-3, betas=(0.9, 0.999))

epoch = 10
loss_hist = []
test_loss_hist = []
counter = 0

#>> Training Loop >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
result=[]
best_acc = 0.0
best_acc_std = 0.0
iter_max = 10

for e in range(epoch):
    rnet.train()
    it=0
    train_loss_list=[]
    train_acc_list=[]
    
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device).permute((1,0,*[i+2 for i in range(inputs.ndim-2)])).to(torch.float), targets.to(device)
        inputs=torch.squeeze(inputs) #Removing unnecessary channel dimensions
        spk2_rec, mem = rnet.forward(inputs)
        print(mem.shape)
        loss:torch.Tensor = criterion(mem, targets)
        loss.backward()
        optim.step()
        optim.zero_grad()
        train_loss_list.append(loss.item())

        train_acc_list.append(calculate_accuracy(mem,targets))

        print(f"Epoch [{e+1}/{epoch}], Step [{batch_idx}/{len(train_loader)}], Loss: {loss.item():.4f}")

        it+=1
        if iter_max>0 and it>iter_max:
            break



torch.Size([112, 128, 20])


RuntimeError: "nll_loss_forward_reduce_cuda_kernel_2d_index" not implemented for 'Int'